# 포스팅 DB화 및 커밋 자동화

In [1]:
from pathlib import Path
import subprocess
import boto3
import yaml
import os


def extract_front_matter_and_content_from_markdown(post_path):
    # 마크다운 프론트 매터와 컨텐츠를 추출함
    with open(post_path, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() == '---':
            end_line_index = lines[1:].index('---\n') + 1
            front_matter = ''.join(lines[1:end_line_index])
            content = ''.join(lines[end_line_index + 1:])
            return yaml.safe_load(front_matter), content
    return None, None

def get_changed_new_or_deleted_md_files(repo_path):
    result = subprocess.run(
        ['git', '-C', repo_path, 'status', '--porcelain=v1'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    files_status = []
    for line in result.stdout.splitlines():
        status, file_path = line[:2].strip(), line[3:].strip().replace('"', '')
        print(status, file_path)
        if status == '??' and os.path.isdir(os.path.join(repo_path, file_path)):
            directory_path = os.path.join(repo_path, file_path)
            md_files = [f for f in os.listdir(directory_path) if f.endswith('.md')]
            # 각 .md 파일에 대해 원하는 작업 수행
            for md_file in md_files:
                md_file_path = os.path.join(directory_path, md_file)
                files_status.append(['untracked', md_file_path.replace(git_repo_path, '')])
        
        if file_path.endswith('.md'):
            file_action = 'modified' if status == 'M' else 'added' if status == 'A' else 'deleted' if status == 'D' else 'untracked'
            files_status.append([file_action, file_path])

    return files_status


# 깃 저장소의 경로
git_repo_path = '/Users/6mini/post/'
md_files_status = get_changed_new_or_deleted_md_files(git_repo_path)

# DynamoDB 설정
dynamodb = boto3.resource('dynamodb')

# 테이블 참조
post_table = dynamodb.Table('Posts')
category_table = dynamodb.Table('Categories')


if md_files_status:
    # 커밋 메세지용 리스트
    commit_slug_list = []
    
    for status, file_name in md_files_status:
        print([status, file_name])
        file_path = Path(git_repo_path) / file_name
        print(f'Processing file: {file_path}')
        
        post_path = str(file_name).replace(git_repo_path, '')
        main_category, sub_category, slug = post_path.split('/')
        
        main_category = main_category[2:]
        sub_category = sub_category[3:]
        slug = slug[4:].replace('.md', '')
        
        print("main_category:", main_category)
        print("sub_category:", sub_category)
        print("slug:", slug)
        
        # 새롭거나 수정이면 추가
        if status != 'deleted':
            # 함수를 사용하여 프론트매터와 내용 추출
            front_matter, content = extract_front_matter_and_content_from_markdown(file_path)

            title = front_matter['title']
            description = front_matter['description']
            date = front_matter['date']
            tags = front_matter['tags']

            print(f"title:", title)
            print(f"description:", description)
            print(f"date:", date)
            print(f"tags:", tags)
            
            # 삽입
            response = post_table.put_item(
                Item={
                    'post_id': slug,
                    'title': title,
                    'description': description,
                    'content': content,
                    'date': date,
                    'main_category': main_category,
                    'sub_category': sub_category,
                    'tags': tags
                }
            )

            # 대분류와 관련된 서브 카테고리 가져오기
            response = category_table.get_item(
                Key={
                    'main_category': main_category
                }
            )

            # 대분류가 테이블에 존재하는지 확인
            if 'Item' in response:
                # 해당 대분류에 서브 카테고리가 이미 존재하는지 확인
                if sub_category not in response['Item'].get('sub_categories', []):
                    # 존재하지 않으면 서브 카테고리 추가
                    updated_sub_categories = response['Item']['sub_categories']
                    updated_sub_categories.append(sub_category)
                    category_table.update_item(
                        Key={
                            'main_category': main_category
                        },
                        UpdateExpression="SET sub_categories = :s",
                        ExpressionAttributeValues={
                            ':s': updated_sub_categories
                        }
                    )
            else:
                # 대분류가 존재하지 않으면 새로운 대분류와 서브 카테고리 추가
                category_table.put_item(
                    Item={
                        'main_category': main_category,
                        'sub_categories': [sub_category]
                    }
                )
        
        # 제거면 삭제
        else:
            # DynamoDB에서 해당 항목 삭제
            response = post_table.delete_item(
                Key={
                    'post_id': slug
                }
            )
            print(f"Deleted post with slug: {slug}")
        
        # 처리 후 메세지 리스트 추가
        commit_slug_list.append(f"{status} {slug}")
    
    # 커밋
    if commit_slug_list:
        commit_message = ', '.join(commit_slug_list)
        try:
            # 모든 변경사항 추가
            subprocess.run(f"git add .", check=True, shell=True)

            # 커밋
            subprocess.run(f"git commit -m '{commit_message}'", check=True, shell=True)

            # 푸시
            subprocess.run("git push", check=True, shell=True)
            print("Commit and push successful.")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred: {e}")

M ADD_POST.ipynb
?? 2-tech/02-data engineering/002-web-crawling-ip-bypass-with-ec2.md
['untracked', '2-tech/02-data engineering/002-web-crawling-ip-bypass-with-ec2.md']
Processing file: /Users/6mini/post/2-tech/02-data engineering/002-web-crawling-ip-bypass-with-ec2.md
main_category: tech
sub_category: data engineering
slug: web-crawling-ip-bypass-with-ec2
title: 웹 크롤링 IP 차단 우회 방법론: AWS EC2 재부팅을 통한 IP 변경 및 연속 수집 자동화 전략
description: Amazon Web Services(AWS)의 Elastic Compute Cloud(EC2)를 이용하여 웹 크롤링 시 발생하는 IP 차단 문제를 우회하고, 크롤링 작업을 자동화하는 방법을 단계별로 설명한다. EC2 인스턴스 재부팅을 통한 IP 변경, AWS S3와 람다(Lambda) 함수를 사용한 자동 재시작 및 종료 제어 방법을 포함한다.
date: 2024-02-15
tags: ['크롤링(Crawling)', '스크래핑(Scraping)', 'IP 차단 우회', '자동화(Automation)', 'AWS', '람다(Lambda)', 'EC2(Elastic Compute Cloud)', 'S3(Simple Storage Service)']
[main 3932879] untracked web-crawling-ip-bypass-with-ec2
 2 files changed, 648 insertions(+), 13 deletions(-)
 create mode 100644 2-tech/02-data engineering/002-web-crawling-ip-bypass-with-ec2.md
Comm

To https://github.com/6mini/post.git
   5f15328..3932879  main -> main
